一部動畫的id會在參數中以`sn`表示  
以[https://ani.gamer.com.tw/animeVideo.php?sn=34142](https://ani.gamer.com.tw/animeVideo.php?sn=34142)為例  
`sn`是34142

In [ ]:
sn = '34142'

先看到了`playlist.m3u8`

In [4]:
! ffplay -i https://bahamut.akamaized.net/ad/welcome_to_anigamer/playlist.m3u8

ffplay version n4.4.2-4-g944ee75106-20220909 Copyright (c) 2003-2021 the FFmpeg developers
  built with gcc 12.1.0 (crosstool-NG 1.25.0.55_3defb7b)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --disable-frei0r --enable-libgme --enable-libkvazaar --enable-libass --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librist --enable-libssh --enable-libtheora --enable-libvpx --enable-li

但顯然這個播放清單是歡迎畫面的，不是動畫本體

觀察有可能的API
- `getdeviceid.php`
- `video.php`
- `token.php`

In [4]:
import requests

header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70', 'referer': 'https://ani.gamer.com.tw/animeVideo.php', 'origin': 'https://ani.gamer.com.tw'}
session = requests.session()
session.headers.update(header)

開始廣告之後會發現一個API呼叫
- videoCastcishu.php?s=...&sn=37780

廣告結束後也有一個呼叫
- videoCastcishu.php?s=...&sn=37780&ad=end

然後發現了一個呼叫`m3u8.php`裡面取得了m3u8的網址
- https://ani.gamer.com.tw/ajax/m3u8.php?sn=37780&device=0052f38c1634575e5e81f13b1daef008c6ae921bc6415a7d660cea1c4571

裡面包含了deviceid，就先預設這是必要參數，傳回內容是一個m3u8的網址

In [8]:
deviceid_res = session.get('https://ani.gamer.com.tw/ajax/getdeviceid.php')
deviceid_res.raise_for_status()
deviceid = json.loads(deviceid_res.text)['deviceid']
print(deviceid)

03529218927ff1e8f79bda3a7f6e13b907559f1e4c6fac29660086f77906


In [30]:
import functions
ad_data = functions.get_major_ad()
session.cookies.update(ad_data['cookie'])
print('start ad')
session.get(f'https://ani.gamer.com.tw/ajax/videoCastcishu.php?s={ad_data["adsid"]}&sn={sn}')
ad_countdown = 25
for countdown in range(ad_countdown):
    print(f'ad {ad_countdown - countdown}s remaining', end='\r')
    time.sleep(1)

session.get(f'https://ani.gamer.com.tw/ajax/videoCastcishu.php?s={ad_data["adsid"]}&sn={sn}&ad=end')
print('end ad')

start ad
end adremainingg


In [12]:
m3u8_php_res = session.get(f'https://ani.gamer.com.tw/ajax/m3u8.php?sn={sn}&device={deviceid}')
m3u8_php_res.raise_for_status()
try:
    playlist_basic_url = json.loads(m3u8_php_res.text)['src']
except Exception as ex:
    print(m3u8_php_res.text)
    print('failed to load m3u')
print(playlist_basic_url)

https://bahamut.akamaized.net/1133666c9d8955814ff4e2b58138949867b3c796/playlist_guest.m3u8?hdnts=exp%3D1711314264%7Edata%3D810e6cdf2177d96%3A37668%3A0%3A1%3A6868682f%7Eacl%3D%2F1133666c9d8955814ff4e2b58138949867b3c796%2Fplaylist_guest.m3u8%21%2F1133666c9d8955814ff4e2b58138949867b3c796%2F360p%2F%2A%7Ehmac%3D042f852a4b09de90298ab49cc73d6cb5aba7fab5244f7c5e7e3089c8ec394e2a


In [14]:
meta_base = os.path.dirname(playlist_basic_url) + '/'
playlist_basic_res = requests.get(playlist_basic_url, headers=header)
playlist_basic_res.raise_for_status()

print(playlist_basic_res.text)

#EXTM3U
#EXT-X-VERSION:3
#EXT-X-STREAM-INF:BANDWIDTH=400000,RESOLUTION=640x360
360p/hdntl=exp=1711397104~acl=%2f1133666c9d8955814ff4e2b58138949867b3c796%2fplaylist_guest.m3u8!%2f1133666c9d8955814ff4e2b58138949867b3c796%2f360p%2f*~data=hdntl,810e6cdf2177d96%3a37668%3a0%3a1%3a6868682f~hmac=02975b3b908d7e85db72d101f115bcbb0f94309d2c2b7e4ec62e280ab4935913/chunklist_b400000.m3u8



In [16]:
playlist_basic = playlist_basic_res.text.split('\n')
resolutions_metadata = []
for i in range(len(playlist_basic)):
    line = playlist_basic[i]
    if line.startswith('#EXT-X-STREAM-INF'):
        resolutions_metadata.append({'info': line[18:], 'url': playlist_basic[i+1]})

In [19]:
print(resolutions_metadata)

[{'info': 'BANDWIDTH=400000,RESOLUTION=640x360', 'url': '360p/hdntl=exp=1711397104~acl=%2f1133666c9d8955814ff4e2b58138949867b3c796%2fplaylist_guest.m3u8!%2f1133666c9d8955814ff4e2b58138949867b3c796%2f360p%2f*~data=hdntl,810e6cdf2177d96%3a37668%3a0%3a1%3a6868682f~hmac=02975b3b908d7e85db72d101f115bcbb0f94309d2c2b7e4ec62e280ab4935913/chunklist_b400000.m3u8'}]


In [17]:
for j in range(len(resolutions_metadata)):
    print('#%s: %s' % (j, resolutions_metadata[j]['info']))

#0: BANDWIDTH=400000,RESOLUTION=640x360


In [18]:
selected_resolution = int(input('select a resolution: '))

In [21]:
resolution_number = resolutions_metadata[selected_resolution]['info'].rsplit('=', 1)[1]
folder_name = f'{sn}_{resolution_number}'
os.makedirs(os.path.join('Download', folder_name), exist_ok=True)
os.chdir(os.path.join('Download', folder_name))

In [23]:
chunklist_res = requests.get(meta_base + resolutions_metadata[selected_resolution]['url'], headers=header)
print(chunklist_res.text)

#EXTM3U
#EXT-X-VERSION:3
#EXT-X-TARGETDURATION:10
#EXT-X-MEDIA-SEQUENCE:0
#EXT-X-KEY:METHOD=AES-128,URI="key_b400000.m3u8key",IV=0xa51891f1791c412dfe0783b8e8ded649
#EXTINF:10.427089,
media_b400000_0.ts
#EXTINF:10.427078,
media_b400000_1.ts
#EXTINF:10.427089,
media_b400000_2.ts
#EXTINF:10.427078,
media_b400000_3.ts
#EXTINF:10.427089,
media_b400000_4.ts
#EXTINF:10.427078,
media_b400000_5.ts
#EXTINF:10.427089,
media_b400000_6.ts
#EXTINF:10.427078,
media_b400000_7.ts
#EXTINF:10.427089,
media_b400000_8.ts
#EXTINF:10.427078,
media_b400000_9.ts
#EXTINF:10.427089,
media_b400000_10.ts
#EXTINF:10.427078,
media_b400000_11.ts
#EXTINF:10.427089,
media_b400000_12.ts
#EXTINF:10.427078,
media_b400000_13.ts
#EXTINF:10.427089,
media_b400000_14.ts
#EXTINF:10.427078,
media_b400000_15.ts
#EXTINF:10.427089,
media_b400000_16.ts
#EXTINF:10.427078,
media_b400000_17.ts
#EXTINF:10.427089,
media_b400000_18.ts
#EXTINF:10.427078,
media_b400000_19.ts
#EXTINF:10.427089,
media_b400000_20.ts
#EXTINF:10.427078,
media_b4

In [24]:
chunklist_filename = folder_name + '.m3u8'
with open(chunklist_filename, 'wb') as chunklist_file:
    for chunk in chunklist_res:
        chunklist_file.write(chunk)

In [25]:
chunks_base = meta_base + resolutions_metadata[selected_resolution]['url'].rsplit('/', 1)[0] + '/'

In [26]:
import multiple_thread_downloading
chunklist = chunklist_res.text.split('\n')
mtd_worker = multiple_thread_downloading.mtd(header, chunks_base, chunklist_res.text.count('.ts'))

In [27]:
for k in range(len(chunklist)):
    line = chunklist[k]
    if line.startswith('#EXTINF'):
        ts_name = chunklist[k+1]
        # push
        mtd_worker.push(ts_name)
    elif line.startswith('#EXT-X-KEY'):
        key_name = re.match('.*URI="(.*)".*$', line).group(1)
        # push
        mtd_worker.push(key_name)

In [28]:
mtd_worker.join()

In [29]:
os.system('ffmpeg -allowed_extensions ALL -i %s -c copy %s.mp4' % (chunklist_filename, folder_name))

0